In [1]:
import uproot
import numpy as np
import matplotlib.pyplot as plt
import os
import re
import matplotlib
import time

from scipy import optimize
from matplotlib.backends.backend_pdf import PdfPages

In [2]:
matplotlib.use('pdf')
#plt.rcParams['figure.dpi'] = 300

In [ ]:
filename = '../Geant4Model/build/Output/output_test.root'
file = uproot.open(filename)

names = np.array(file.keys())
times = names[:len(names) // 3]
cover_hists = names[len(names) // 3::2]

In [3]:
def GetCarpetSignal(hist):
    return hist[hist.shape[0] // 2 - 10:hist.shape[0] // 2 + 10, hist.shape[0] // 2 - 10:hist.shape[0] // 2 + 10]

def VEM(hist):
    filt = (hist < 0.5)
    hist[filt] = 0.001
    hist = (1 + np.log(hist/8)/np.log(1.12)) // 1
    hist[hist < 0] = 0
    #hist[filt] = -1

    rel_p = 8*1.12**(hist - 1)
    rel_p[rel_p < 8] = 0

    return rel_p

In [4]:
def fit_coeffs(row_ind, row_centres, row_weights, col_ind, col_centres, col_weights):

    row_polyfit = np.polynomial.Polynomial.fit(row_ind, row_centres, deg=1, w=row_weights).convert().coef
    col_polyfit = np.polynomial.Polynomial.fit(col_ind, col_centres, deg=1, w=col_weights).convert().coef

    b_row = row_polyfit[0]
    k_row = row_polyfit[-1]

    b_col = col_polyfit[0]
    k_col = col_polyfit[-1]

    row_errors = (row_centres - k_row*row_ind - b_row)**2
    col_errors = (col_centres - k_col*col_ind - b_col)**2

    if np.any(row_errors > 5**2):
        
        worst_point = np.argmax(row_errors)
        
        row_ind = np.delete(row_ind, worst_point)
        row_centres = np.delete(row_centres, worst_point)
        row_weights = np.delete(row_weights, worst_point)

        row_polyfit = np.polynomial.Polynomial.fit(row_ind, row_centres, 1, w=row_weights).convert().coef

        b_row = row_polyfit[0]
        k_row = row_polyfit[-1]

    if np.any(col_errors > 5**2):
        
        worst_point = np.argmax(col_errors)
        
        col_ind = np.delete(col_ind, worst_point)
        col_centres = np.delete(col_centres, worst_point)
        col_weights = np.delete(col_weights, worst_point)

        col_polyfit = np.polynomial.Polynomial.fit(col_ind, col_centres, 1, w=col_weights).convert().coef

        b_col = col_polyfit[0]
        k_col = col_polyfit[-1]

    return np.array([[b_row, k_row], [b_col, k_col]])

def get_xy_fits(hist):

    non_zero_cols = np.count_nonzero(np.count_nonzero(hist, axis=0))
    non_zero_rows = np.count_nonzero(np.count_nonzero(hist, axis=1))
    
    if ((non_zero_cols >= 2) and (non_zero_rows >= 2)):
    
        n_rows = hist.shape[0]
        n_cols = hist.shape[1]
    
        row_ind = np.arange(0, n_rows)
        col_ind = np.arange(0, n_cols)
    
        row_weights = np.zeros(n_rows)
        row_centres = np.zeros(n_rows)
        
        col_weights = np.zeros(n_cols)
        col_centres = np.zeros(n_cols)
    
        for i in range(n_rows):
            row = hist[i]
            
            weight = np.sum(row)

            maxima = np.argwhere(row == row.max())
            centre = np.mean(maxima)
            
            row_weights[i] = weight
            row_centres[i] = centre
        for i in range(n_cols):
    
            col = hist[:, i]
            
            weight = np.sum(col)
            
            maxima = np.argwhere(col == col.max())
            centre = np.mean(maxima)
            
            col_weights[i] = weight
            col_centres[i] = centre
    
        coeffs = fit_coeffs(row_ind, row_centres, row_weights, col_ind, col_centres, col_weights)

        b_row = coeffs[0, 0]
        k_row = coeffs[0, 1]

        b_col = coeffs[1, 0]
        k_col = coeffs[1, 1]

        if (k_col*k_row == 1):
            col0 = np.mean(col_ind[np.nonzero(col_weights)])
            row0 = k_col*col0 + b_col
        else:
            col0 = (k_row*b_col + b_row) / (1-k_row*k_col)
            row0 = k_col*col0 + b_col
        
        return [row0, col0]
    
    else:

        #empty_poly = np.poly1d(9.5)
        
        return [9.5, 9.5]

In [ ]:
cover_signals = np.zeros((len(cover_hists), 20, 20))

for i, hist in enumerate(cover_hists):
    cov_hist = np.copy(np.flip(file[hist].values().T, 0))

    cover_signals[i] = GetCarpetSignal(cov_hist)

In [ ]:
ax = plt.axes()
ax.set_facecolor('k')

plt.imshow(cover_signals[0], cmap='turbo')

row0, col0 = get_xy_fits(VEM(cover_signals[0]))

plt.plot(col0, row0, 'mo')

In [ ]:
pdf = PdfPages("100_Showers_intersection_reconstruction.pdf")

for signal in cover_signals:

    sinal_copy = np.copy(signal)

    sig = VEM(sinal_copy)

    plt.imshow(signal, cmap='turbo')

    row0, col0 = get_xy_fits(sig)

    plt.plot(col0, row0, 'mo')

    pdf.savefig()
    plt.close('all')

pdf.close()

In [5]:
def num_sort(test_string):
    return list(map(int, re.findall(r'\d+', test_string)))[0]

In [6]:
path = '../NKG_fit/proton_data/'

filenames = [path + filename for filename in os.listdir(path)]
filenames.sort(key=num_sort)

In [23]:
high_energy_files = filenames[18:]

In [24]:
high_energy_files

['../NKG_fit/proton_data/output18.root',
 '../NKG_fit/proton_data/output19.root',
 '../NKG_fit/proton_data/output20.root',
 '../NKG_fit/proton_data/output21.root',
 '../NKG_fit/proton_data/output22.root',
 '../NKG_fit/proton_data/output23.root',
 '../NKG_fit/proton_data/output24.root',
 '../NKG_fit/proton_data/output25.root',
 '../NKG_fit/proton_data/output26.root',
 '../NKG_fit/proton_data/output27.root',
 '../NKG_fit/proton_data/output28.root']

In [25]:
pdf = PdfPages('proton_data.pdf')

for filename in high_energy_files:

    print(filename)

    file = uproot.open(filename)

    names = np.array(file.keys())
    times = names[:len(names) // 3]
    cover_hists = names[len(names) // 3::2]
    
    cover_signals = np.zeros((len(cover_hists), 20, 20))
    
    for i, hist in enumerate(cover_hists):
        cov_hist = np.copy(np.flip(file[hist].values().T, 0))
        cover_signals[i] = GetCarpetSignal(cov_hist)
    
    for i, signal in enumerate(cover_signals):
        sinal_copy = np.copy(signal)
    
        sig = VEM(sinal_copy)

        row0, col0 = get_xy_fits(sig)
        if ((row0 != 9.5) or (col0 != 9.5)):
    
            plt.imshow(signal, cmap='turbo')
        
            plt.plot(col0, row0, 'mo')

            for j in range(sig.shape[0]):
                for k in range(sig.shape[1]):
                        text = plt.text(k, j, int(signal[j, k]), ha="center", va="center", color="w", fontsize=5)

            EAS_par = cover_hists[i].split('_')

            theta, phi, x0, y0 = np.array(EAS_par[1:5]).astype(float)

            delta_r = 10*np.tan(np.radians(theta))
            delta_x = delta_r*np.cos(np.radians(phi))
            delta_y = delta_r*np.sin(np.radians(phi))

            true_x = np.round(x0 + delta_x, 3)
            true_y = np.round(y0 + delta_y, 3)

            plt.title('x = ' + str(true_x) + '; y = ' + str(true_y))
        
            pdf.savefig()
            plt.close('all')

pdf.close()

../NKG_fit/proton_data/output18.root
../NKG_fit/proton_data/output19.root
../NKG_fit/proton_data/output20.root
../NKG_fit/proton_data/output21.root
../NKG_fit/proton_data/output22.root
../NKG_fit/proton_data/output23.root
../NKG_fit/proton_data/output24.root
../NKG_fit/proton_data/output25.root
../NKG_fit/proton_data/output26.root
../NKG_fit/proton_data/output27.root
../NKG_fit/proton_data/output28.root


# Mexican Hat

In [ ]:
def hat_func(params, hist):

    y, x = params
    
    n_rows = hist.shape[0]
    n_cols = hist.shape[1]

    f = 0

    for i in range(n_rows):
        for j in range(n_cols):
            
            row = i
            column = j

            t_ij_sq = (row - y)**2 + (column - x)**2

            entry = hist[row, column]*(2 - t_ij_sq)*np.exp(-t_ij_sq/2)

            f += entry

    return -f

In [ ]:
T_max = 25
T_min = 0.01

def get_annealing_xy(hist):

    n_rows = hist.shape[0]
    n_cols = hist.shape[1]

    T = T_max
    y = 9.5
    x = 9.5
    E = hat_func([y, x], hist)

    for i in range(1, 1_000):
        
        x_rand = np.random.uniform(-0.5, n_cols - 0.5)
        y_rand = np.random.uniform(-0.5, n_rows - 0.5)
        E_rand = hat_func([y_rand, x_rand], hist)
        deltaE = E_rand - E

        if deltaE < 0:
            
            y = y_rand
            x = x_rand
            E = E_rand
            
        else:
            
            prob = np.exp(-deltaE/T)
            change_prob = np.random.uniform()

            if change_prob <= prob:

                y = y_rand
                x = x_rand
                E = E_rand

        T = T_max/np.log(1 + i)

        if T <= T_min: break

    return [y, x]
            

In [ ]:
hist = VEM(cover_signals[0])

#initial_guess = np.unravel_index(np.argmax(hist, axis=None), hist.shape)

row0, col0 = get_annealing_xy(hist)

print(row0, col0)

In [ ]:
pdf = PdfPages("100_Showers_hat_reconstruction.pdf")

for signal in cover_signals:

    sinal_copy = np.copy(signal)

    sig = VEM(sinal_copy)

    plt.imshow(signal, cmap='turbo')

    y0, x0 = get_annealing_xy(sig)

    plt.plot(x0, y0, 'mo')

    pdf.savefig()
    plt.close('all')

pdf.close()

In [ ]:
start = time.time()

for signal in cover_signals:
    get_xy_fits(signal)

end = time.time()

(end-start)*1000